In [1]:
#pip install transformers

In [2]:
#pip install torch torchvision torchaudio

In [15]:
import torch
print(torch.__version__)

2.5.1+cpu


In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Binary classification


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
import pandas as pd

In [25]:
from sklearn.model_selection import train_test_split

In [27]:
# Load dataset
df = pd.read_csv("moviereviews2.tsv", sep="\t")

In [29]:
# Map sentiment labels to integers
label_mapping = {"pos": 1, "neg": 0}  # Define the mapping
df['label'] = df['label'].map(label_mapping)


In [31]:
df

,label,review
0,1,I loved this movie and will watch it again. Or...
1,1,"A warm, touching movie that has a fantasy-like..."
2,1,I was not expecting the powerful filmmaking ex...
3,0,"This so-called ""documentary"" tries to tell tha..."
4,1,This show has been my escape from reality for ...
...,...,...
5995,1,"Of the three remakes of this plot, I like them..."
5996,0,Poor Whoopi Goldberg. Imagine her at a friend'...
5997,0,"Honestly before I watched this movie, I had he..."
5998,1,This movie is essentially shot on a hand held ...


In [33]:
# Check for unmapped values
if df['label'].isnull().any():
    print("Unmapped values detected. Cleaning them...")
    print(df[df['label'].isnull()])

In [35]:
# Clean data
df = df.dropna(subset=['review', 'label'])  # Drop missing rows
df['label'] = df['label'].astype(int)  # Ensure labels are integers

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22364\2524293877.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)  # Ensure labels are integers


In [37]:
review = df["review"]
label = df["label"]


In [39]:
train_texts, test_texts, train_labels, test_labels = train_test_split(review, label, test_size=0.2, random_state=42)


In [41]:
train_texts = train_texts.tolist()
train_labels = train_labels.tolist()

In [54]:
#print("Example Train Texts:", train_texts[:10])
#print("Example Train Labels:", train_labels[:10])

In [45]:
# Tokenize all texts
max_length = 128  # Maximum sequence length
train_tokens = tokenizer(train_texts, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")

# Convert labels to tensors
train_labels_tensor = torch.tensor(train_labels)

In [47]:
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset
dataset = TensorDataset(train_tokens["input_ids"], train_tokens["attention_mask"], train_labels_tensor)

# Training DataLoader
#train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], train_labels_tensor)
#train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Testing DataLoader
#test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], test_labels_tensor)
#test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Create a DataLoader
batch_size = 16
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [52]:
from transformers import AutoModelForSequenceClassification
from torch.nn.functional import cross_entropy

# Load pre-trained BERT for binary classification
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Example training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

for epoch in range(3):  # Number of epochs
    for batch in data_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # Cross-entropy loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")


Epoch 0, Loss: 0.6423455476760864
Epoch 0, Loss: 0.6798015236854553


KeyboardInterrupt: 

In [244]:
# Save the model
model.save_pretrained("sentiment_model2")

# Save the tokenizer
tokenizer.save_pretrained("sentiment_model2")


('sentiment_model2\\tokenizer_config.json',
 'sentiment_model2\\special_tokens_map.json',
 'sentiment_model2\\vocab.txt',
 'sentiment_model2\\added_tokens.json',
 'sentiment_model2\\tokenizer.json')

In [276]:
validation_texts=test_texts.tolist()
#print(type(validation_texts))
#print(validation_texts)[:2]
#print(validation_texts)

validation_labels=test_labels.tolist()
#print(type(validation_labels))
#print(validation_labels)[:2]
#print(validation_labels)

<class 'list'>
['Perhaps I couldn\'t find the DVD menu selection for PLOT: ON OFF. Clearly, the default is OFF. When the end credits began to roll, I couldn\'t believe that was it. Like our poor, but beautiful protagonist, I felt used, dirty, cheap....<br /><br />The characters were drawn in very broad strokes and the writer\'s disdain for wealthy Thatcherites was all to apparent. I consider myself a "Roosevelt Democrat", but would appreciate a bit more subtlety.<br /><br />Of course, the problem could be with me. I see that many others seem to find some meaning or message in this picture. Alas, not I. <br /><br />The only thing that kept me from giving this a "1" was the nice scenery, human and plant.', "Wow this was a movie was completely captivating I could not believe that I started awake so late to watch it but it came on late ounce I started watching I couldn't stop it had a full range of very good cast members wow even Eartha Kitt and Ruby Dee Forrest Whittaker and James Earl Jo

In [278]:
from sklearn.metrics import accuracy_score

# Example validation data

#validation_texts=test_texts
#validation_labels=test_labels
#validation_texts = ["This was an amazing film!", "Terrible experience."]
#validation_labels = [1, 0]  # Ground truth labels

# Tokenize the validation texts
val_tokens = tokenizer(validation_texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
#print(val_tokens)


{'input_ids': tensor([[  101,  3383,  1045,  ...,  2033,  1012,   102],
        [  101, 10166,  2023,  ...,  2381,  2196,   102],
        [  101,  1045,  2034,  ...,  2464,  2070,   102],
        ...,
        [  101, 24003, 16595,  ...,  3110,  2169,   102],
        [  101,  2200,  4298,  ...,     0,     0,     0],
        [  101,  5980, 10555,  ...,  4678,  2122,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [8]:
model = AutoModelForSequenceClassification.from_pretrained("sentiment_model2")

In [13]:
# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(val_tokens['input_ids'], attention_mask=val_tokens['attention_mask'])
    predictions = torch.argmax(outputs.logits, dim=1)  # Get predicted labels

# Calculate accuracy
accuracy = accuracy_score(validation_labels, predictions.numpy())
print(f"Validation Accuracy: {accuracy:.2f}")

NameError: name 'val_tokens' is not defined

In [3]:
# Example input for prediction
#new_text = "I absolutely loved this product!"
#new_text = "I was kinda in two minds about this movie"
new_text = "I watched this movie and I was like mehhhh, sucks"

# Tokenize the input
new_tokens = tokenizer(new_text, max_length=128, padding="max_length", truncation=True, return_tensors="pt")

# Predict
model.eval()
with torch.no_grad():
    output = model(new_tokens['input_ids'], attention_mask=new_tokens['attention_mask'])
    prediction = torch.argmax(output.logits, dim=1)

# Interpret the result
sentiment = "Positive" if prediction.item() == 1 else "Negative"
#print(prediction)
#print(output)
print(f"Sentiment: {sentiment}")


NameError: name 'tokenizer' is not defined